In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
test_data = pd.read_csv(test_file_path, sep='\t', header=None)

train_data.columns = ['class', 'sms']
test_data.columns = ['class', 'sms']

train_labels = train_data.pop('class')
test_labels = test_data.pop('class')

class_map = {'ham':0, 'spam':1}
train_labels = train_labels.map(class_map)
test_labels = test_labels.map(class_map)

train_data = np.array(train_data['sms'])
test_data = np.array(test_data['sms'])

test_labels = np.array(test_labels)
train_labels = np.array(train_labels)

In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization()
vectorize_layer.adapt(train_data)
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())

In [ ]:
model = keras.Sequential([
    vectorize_layer,
    keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ['acc'])

In [ ]:
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.25)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict(np.array([pred_text]))
  prediction = [pred[0][0], 'ham'] if pred[0][0] < 0.5 else [pred[0][0], 'spam']
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
